# Storage and Retrieval

* On the most fundamental level, a database needs to do two things: store data and retrieve it back when required
* In Chapter 2, we discussed data models and query languages: format in which we give db the data and mechanism by which we ask for it again later
* In this chapter, we discuss from db's point of view: how we can store data that we're given and how we can find it again when asked for it
* In order to tune a storage engine for our workload, we need to have a rough idea of what it's doing under the hood
* There is a big difference between storage engines optimised for transactional workload vs analytics
* We examine two types of storage engines: log-structured and page-oriented

## Data Structures That Power Databases

* Consider the simplest possible db - key value pairs stored in a csv that can be written to and fetched using bash functions (let's call them set and get)
* Every call to set function appends to the end of the file - so if you update key several times, the old versions of value are not overwritten. We need to look at the last occurance of key in the file to get latest value
* The set performance of this simple db is good since appending is very efficient. Many real world dbs use a similar principle by maintaining a append only log, however handle more complexities such as concurrency control, reclaiming list disk space to avoid evergrowing logs and handling errors like partially written logs.
* However, the get function does not have good performance if the number of records is large in database. It has O(n) complexity which means it will take double the time if number of records double 
* For efficiency, we need a different data structure: index
* Index is an additional structure that is derived from primary data - this does not impact content of db but only impacts the performance of queries
* Any kind of index will slow down writes (as compared to simply appending at end of file) since index also needs to be updated during writes
* Hence, there is a trade-off in storage systems between indexes to speed up reads vs slowing down writes
* Therefore, dbs will NOT index everything by default but depend on application developers or db admin to set it up

**Hash Indexes**
 * Most commonly indexed data is for key-value data (although you can index other kind of data as well)
 * Key-value stores are similar to dictionary type found in most programming languages, implemented with hashmaps
 * Assume if our data storage consists only of appending to a file. The simplest indexing strategy would be to keep a in-memory hashmap where every key is mapped to a byte offset in the data file (location where value can be found)
 * When a new key-value pair is appended to the file, we update the hashmap (insert new keys+update existing ones)
 * This is essentially how Bitcask (default storage engine in Riak db)
 * The main requirement is that keys should fit in memory. Even if values take more space than available memory, we can load them from disk with one disk seek. If that part of data file is already in the cache, then the read does not require any disk I/O
 * Such storage engine (like Bitcask) is well suited were value of key is updated frequently. Example: Update number of view/plays for any video. Here the number of keys are limited, but the writes per key are heavy
 * Now consider the issue of everygrowing log file, given we only append to it for writes
 * A good solution is to break the log into segments of certain size by closing segment file when it reaches a certain size and making subsequent writes to new segment file
 * We perform compaction on these segments, where we throw away duplicate keys in the log and keep only recent update for each key
 * Since compaction often makes segments smaller, we can also merge several segments together at the same time and write in a new segment file (segments are never modified once written)
 * Merging and compaction can be done in background thread, while still serving reads and writes
 * After merging process is compelte, we switch read requests to using the new merged segment and old segment files can be deleted
 * Each segment has its own in-memory hash table, mapping keys to file offsets. To find the value for a key, we check most recent segment's hashmap; if value is not present we check second segment's hash table and so on. Merging process reduces number of hashmaps for lookup
 * In order to make this simple idea work in practice, a few more elements are necessary
   - File format: Binary format is faster and simpler to use than CSV
   - Deleting records: To delete a key/value pair, we append a special deletion record to data file called (called as tombstone). During merging, the tombstone tells the merging process to discard any previous values for the deleted key
   - Crash recovery: In order to fasten the process of recovery if db is restarted, a snapshot of each segment's hashmap is stored on disk. This can be loaded in memory quickly
   - Partially written records: If the db crashes, there may be halfway appends in the log. Bitcask files include checksums, allowing such corrupted parts of the log to be detected and ignored
   - Concurrency control: While writing (appending to logs) happens in a strictly sequential manner, reads can be done concurrently by multiple threads
 * Appending files v/s updating file in place
   - Appending and segment merging are sequential write operations, which are generally faster than random writes. This is especially true on magnetic spinning-disk hard drives and to some extend on flash-based solid state drives (SSDs)
   - Concurrency and crash recovery are simpler if segment files are immutable
   - Merging old segments avoid problem of fragmentation of data files over time
 * Limitations of Hash Index:
   - Hash table must fit into memory (won't work if we have large number of keys). On-disk hashmap does not work well as it has lot of random access I/O
   - Range queries are not efficient - each key in the range has to be individually looked up

**SSTables and LSM-Trees**
 * In the segment files, the order of keys does not matter except for the fact that latest values are considered
 * Now consider a change where we require the sequence of key-value pairs is sorted by key
 * This format is called Sorted String Table or SSTable. It also requires that each key appears only once within each merged segment file (which the compaction process already ensures)
 * SSTables have several advantages over log segments
   - Merging segments is simple and efficient, even if files are bigger than available memory. The approach is similar to mergesort algorithm where we start reading input files side by side, lok at the first key in each file and copy the lowest key to the output file and repeat. We retain only the most recent value for each key while merging.
   - For reading, all the keys are not required in the index in memory. Even if some keys exist, we can find other ones since we know the offset range within which the keys should fall due to sorting. Example: If we know offsets for keys handbag and handsome, then we know handiwork should fall between these two offsets
   - One key for every few kilobytes of segment file is sufficient, since key kb can be scanned quickly
   - It is possible to group several key-value pairs in a range into a block and compress it while writing to disk. Each entry of the sparse in-memory index points to the start of the compressed block. Besides saving disk space, compression reduces I/O bandwidth use

***Constructing and maintaining SSTables***
 * Maintaining a sorted structure on disk is possible (B-Trees), however maintaining it in memory is much easier using well-known tree data structures like red-black trees or AVL trees. With these structures, we can insert keys in any order and read them back in sorted order
 * We can make the storage engine work as follows:
   - When a write comes in, add it to a in-memory balanced tree data structure. This in memory tree is sometimes called as memtable
   - When the memtable gets bigger than the threshold, typically a few megabytes - write it out to disk as an SSTable file. New SSTable file becomes the most recent segment of the db
   - To serve a read table, first find the key in memtable, then in most recent on-disk segment and so on
   - From time to time, run merging and compaction to combine segment files and discard overwritten or deleted values
   - In order to handle db crashes where recent writes in memtable could be lost, we can keep a separate log on disk where every write is appended and can be used to restore when required
   
Related Q/A: https://stackoverflow.com/questions/69103575/how-to-maintain-the-sparse-index-in-a-lsm-tree

*Making an LSM-tree out of SSTables*
 * The algorithm described above is used in LevelDB and RocksRB. LevelDB can be used in Riak as an alternative to Bitcask. Similar storage engines are used in Cassandra and Hbase, both inspired by Google's Bibtable paper (which introduced SSTable and memtable)
 * Originally this indexing structure was described by Patrick O'Neil under name Log-Structured Merge-Tree (LSM-Tree)
 * Storage engines that are based on the principle of merging and compacting sorted files are often called LSM storage engines
 * Lucene, an indexing engine for full-text search used by ES and Solr uses a similar method for storing its term dictionary

***Performance Optimisations***
 * LSM-tree algorithm can be slow when looking for keys that dont exist in db since we need to check memtable and all segments before we can be sure that a key does not exist
 * In order to optimise this access, storage engines often use additional Bloom filters. They are a memory-efficient data structure for approximating contents of a set
 * There are different strategies to determine order and timing of how SSTables are compacted and merged
 * Most common options are size-tiered and leveled-compaction
 * In size-tiered: newer and smaller SSTables are successively merged into older and larger SSTables
 * In leveled compaction: key range is split up into smaller SSTables and older data is moved into separate levels, which allows the compaction to proceed more incrementally and use less disk space
 * LevelDB and RocksDB use leveled compaction, HBase uses size-tiered, Cassandra supports both

***B-Trees***
 * The log-structured indexes are gaining acceptance, but the most widely used indexing structure is B-tree
 * While log-structured indexes break db into variable-size segments (typically several mb or more in size) and always write a segment sequentially
 * B-trees break the db into fixed-size blocks or pages (traditionally 4 kb) and read or write one page at a time. This design closely corresponds to underlying hardware as disks are also arranged in fixed-sized blocks
 * Each page can be indentified using an address or location, which allows one page to refer to another (this is like an on-disk pointer)
 * When we want to look up a key in the index, we start with the root page. This root page has keys and references to child pages
 * Each child is responsible for continuous range of keys
 * The number of references to child pages in one page of B-tree is called branching factor
 * If we want to update value of existing key, we first find the leaf page containing the key, change the value in that page and then write the page back to disk (references to page are not changed)
 * If we want to add a new key, we need to find the page whose range encompasses the new key and add it to that page. If there is not enough space to add the new key, the page is split into two half-full pages, and the parent page is updated to account for new pages
 * This algo ensures that tree is balanced (B-tree ith n keys always have depth of O(log n )
 * Most dbs can fit into B-tree that is 3-4 levels deep (4-level tree of 4 KB pages with a branching factor of 500 can store upto 256 TB)
 

***Making B-trees reliable***
 * B-trees assume that overwrite does not change location of page, i.e. we modify files in place as opposed to LSM-trees where we only append to files
 * Overwriting a page on disk requires an actual hardware operation. On magnetic hard drive, this means moving the disk head to the right place, waiting for the right position on the spinning platter to come around and then overwritting the sector with new data.
 * Some operations required several pages to be overwritten - ex: when page is split because of being full during insertion, we need to write two pages during split and overwrite their parent page to update references to children. During such an operation if the db crashes, we can end up with a corrupted index
 * In order to make the db resilient to crashes, B-tree implementations commonly include a wirte-ahead log (WAL or called redo log). This is an append only file which can be used to retore B-tree back to consistent state
 * Concurrency control is required if multiple threads are going to access B-tree at same time during page updates. This is typically done using latches (lightweight locks). LSM trees are simpler in this regard as they do all merging in background without interfering with incoming queries

***B-tree Optimisations***
 * Instead of overwriting pages and maintaining a WAL, some dbs like LMDB us a copy-on-write scheme, where a modified page is written to a new location and new version of parent page is created pointing to new location
 * Instead of storing entire key, we can abbreviate it to save space in pages
 * Pages can be positioned anywhere on disk, there is no requirement to have pages with nearby keys to be nearby on disks. This can be an issue since query could need to scan over large part of key range in sorted order and might need several disk seeks. Therefore, many B-tree implementations try to lay out the tree so that leaf pages appear sequentially on disk. However, maintaining that order is difficult as the tree grows. LSM trees handle this much easily
 * Additional pointers can be added to tree, including siblings apart from children. This allows scanning keys in order without jumping back to parents
 * B-tree variants like fractal trees borrow log-structured ideas to reduce disk seeks (they have nothing to do with fractals)

**Comparing B-Trees and LSM-Trees**
 * As a rule of thumb, LSM-trees are typically faster for writes and B-trees are faster for reads.  LSM-trees are slower for reads as they have to check several different data structures and SSTables at different stages of compaction
 * Both B-trees and Log-structured indexes write data multiple times. This is called write-amplification. In B-trees this is for writing to WAL and once to tree page (and again when pages are split). In log-structured,  its due to repeated compaction and merging of SSTables. Write amplification is particularly bad for SSDs, which wear out after a limited number of writes
 * LSM-trees are able to typically sustain higher write throughput than B-trees, because a) sometimes they have lower write amplification (depending on configs) and b) they sequentially write compact SSTables than having to overwrite several pages in tree. This is especially important on magnetic drives where sequential writes are much faster than random writes
 * LSM trees can be compressed better and often produce smaller files on disk than B-trees. B-trees storage engines leave some disk space unused due to fragmentation (when page is split, some space in page remains unused). Especially with leveled compaction, LSM trees use lower storage overheads
 * Compaction process can sometimes interfere with the performance of ongoing reads and writes (although storage engines perform compaction without affecting concurrent access, disks have limited resources and a request could need to wait while disk finishes an expensive compaction operation

**Other Indexing Structures**
 * So far we discussed key-value indexes, which are like a primary key index in relational model. It is also common to have secondary indexes (keys are not unique). Both B-trees and LSM-trees can be used as secondary indexes
 * If the value is reference to the row (stored elsewhere), it is called a heap file
 * If the extra hope from index to heap file leads to performance issues, it can be desirable to store indexed row directly within an index - this is called clustered index
 * Multi-column indexes are required when we need to query multiple columns of a table (or multiple fields in a document)
 * Most common type of multi-column index is concatenated index which simply combines several fields into one key by appending one column to another
 * In Lucene, a trie-like in-memory index is used to support fuzzy text search

**Keeping everything in memory**
 * Disks (both magnetic and SSDs) are awkward to work with as compared to main memory - data needs to be laid out carefully if we want good performance on reads/writes. This is a trade off since disks are durable (contents are not lost when turned off) and have a lower cost per gb than RAM
 * However, as RAM becomes cheaper we can store entire datasets (when not very large) by potentially distributing them across several machines. This has led to in-memory databases
 * Some in-memory key-value stores like Memcached are intended for caching use only (acceptable for data to be lost when machine is restarted)
 * But other in-memoty dbs aim for durability which can be achieved with special hardware (battery powered RAM), by writing log of changes to disk, by writing periodic snapshots to disk or replicating in-memory state to other machines
 * VoltDB, MemSQL and Oracle TimesTen are in-memory dbs with a relational model
 * RAMCloud is an open-source, in-memory key-value store with durability (log-structured approach for data in memory and also for disk)
 * Redis and Couchbase provide weak durability by writing to disk asynchoronously 
 * The performance advantage of in-memory dbs comes from the fact that they avoid overhead of encoding in-memory data structures in a form that can be written to disk and not necessarily due to the fact that they avoid reading from disk (even in disk-based storage, operating system caches recent blocks in memory)
 * In-memory db architecture could be extended to support datasets larger than available memory by evicting least recently used data from memory to disk and there is not enough memory and loading it back when accessed in future. This still requires indexes to fit in memory (like Bitcask)

**Transaction Processing or Analytics?**
 * Earlier, same dbs were used for both transaction processing (making a sale, placing order)
 * OLTP systems are expected to be highly available and to process transactions with low latency. Therefore, typically db admins are reluctant to allow analysts to run ad hoc queries on OLTP db (Online transaction processing) since these queries are expensive running on large data
 * A data warehouse in contrast is a separate db where analysts can query OLAP (online analytic processing) requirements without affecting OLTPs
 * Data warehouse contains a read-only copy of data in various OLTP systems - data is extracted from OLTP dbs and transformed into a analysis friendly schema using ETL (extract,transform, load)
 * Data model of most data warehouses is relational because SQL is generally good fit for analytical queries
 * On the surface data warehouse and relational OLTP might look similar due to SQL interface, however the underlying systems could be quite different as they are optimised for different query patterns
 * Many vendors focus on supporting either transaction processing OR analytics workloads but not both
 * OLTP systems are typically user-facing and see huge volume of requests as opposed to data warehouses where typically analysts use it. Disk seek time is the bottleneck here. However, each query is demanding requiring millions of records to be scanned in a short time. Disk bandwidth is the bottleneck here.

**Stars and Snowflakes: Schemas for Analytics**
 * Star schema (also called dimensional modeling) is the most commonly used
 * At the center of schema is fact table, where each row of fact table represents an event that occurred at a particular time
 * While each row in fact table represents an event, the dimension tables represent the who, what, where, when, how and why of the event
 * Some of the columns in fact table are attributes, while others are foreign keys to dimension tables
 * Name star schema comes from the fact that fact table is in middle, surrounded by dim tables that connect to it
 * A variation of this template is called snowflake schema, where dimensions are further broken down into subdimensions
 * Essentially, snowflake schemas are more normalised than star schemas, but star schemas are often preferred because they are simpler for analyts to work with

**Column-Oriented Storage**
 * If we have trillions of rows/petabytes of data in fact tables, storing and querying them becomes challenging (dim tables are typically smaller)
 * Most OLTP or document dbs are row-oriented, that is all the values from one row or document are stored next to each other. This is an issue when number of rows is high since they need to be loaded in memory for processing
 * Column-oriented storage instead stores all values from one column together (and not rows)
 * Since only few columns could be used, query only needs to read and parse those columns
 * Further, we can compress data to reduce demands on disk as column values can often be repeatitive
 * Depending on data, different compression techniques can be used. One such effective technique is bitmap encoding
 * Column compression allows more rows from a column to fit in same amount of L1 cache in CPUs
 * Operators like bitwise AND, OR used for bitmap encoding can be designed to directly operate on compressed column data. This is called vectorised processing

**Sort order in Column Storage**
 * In a column store, the order of rows don't matter and can just insert in order of insertion
 * However, we can choose to impose order like we did with SSTables and use that indexing mechanism
 * We cannot independently sort each column as it will then be not possible to know which items belong to which row. Rather the data needs to be sorted on entire row (even though its stored by column)
 * DB admins can choose which columns to sort by, using understanding of query patterns
 * Sorted order can help with compression of columns - if primary sort column does not have many distinct values, it will have long sequences of same repeated values
 * An extension of this idea is to store same data sorted in different ways. Since data across machines needs to be replicated anyway (to recover when one machine fails), this does not add to any additional storage. This was introduced in C-store and adopted in Vertica

**Writing to Column-Oriented Storage**
 * These optimisations make sense in data warehouses since most queries from analysts are read-only
 * The downside is of making writes more difficult
 * Update in place approach like B-trees is not possible with compressed columns. However we can use LSM trees

**Aggregation: Data Cubes and Materialized Views**
 * Materialised view is one way of creating a cache for frequently used aggregates
 * As oppossed to a standard view, materialised view is actual copy of query results written to disk where as virtual view is just a shortcut for writing queries
 * Since any updates to underlying table need to be passed on to materialised view, it can make writes more expensive. Therefore in OLTP dbs, materialised views are avoided. However in read heavy data warehouses, they make more sense
 * Special case of materialised view is called as data cube or OLAP cube. It is a grid of aggregates grouped by different dimensions